In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from keras import layers

In [ ]:
import numpy as np

In [ ]:
(train_examples, validation_examples, test_examples), info = tfds.load(
    'colorectal_histology', 
    with_info=True, 
    as_supervised=True, 
    split=['train[:80%]', 'train[80%:95%]', 'train[95%:]'],
)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return image, label

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_50/feature_vector/5", 
                                            output_shape=[2048], 
                                            trainable=False), 
                             layers.Dense(num_classes, activation='softmax')
])

model.build([None, IMAGE_RES, IMAGE_RES, 3])

model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])

model.fit(train_batches, epochs=10, validation_data=validation_batches)

In [ ]:
score = model.evaluate(train_batches, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])